# 3 Pointers Made against shooting.csv

### Import packages

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
from scipy.stats import pearsonr
import itertools

pd.set_option("display.max_columns", None)

### Set working directory

In [2]:
# Print working directory
cwd = os.getcwd()
print(f'Directory: {cwd}')

# Change working directory
os.chdir('/Users/tyler/OneDrive/Documents/Python/NBA')

# Print working directory
cwd = os.getcwd()
print(f'Directory: {cwd}')

Directory: C:\Users\tyler\OneDrive\Documents\Python\NBA\backend\analysis\3p
Directory: C:\Users\tyler\OneDrive\Documents\Python\NBA


## Exploratory Data Analysis

### Import data

In [3]:
df = pd.read_csv('backend/data/shooting.csv').drop(['Unnamed: 0'], axis=1)

### Basic exploration

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205810 entries, 0 to 205809
Data columns (total 17 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   date      205810 non-null  object 
 1   visitor   205810 non-null  object 
 2   home      205810 non-null  object 
 3   team      205810 non-null  int64  
 4   quarter   205788 non-null  object 
 5   fg        205768 non-null  float64
 6   fga       205768 non-null  float64
 7   fg_perc   205768 non-null  float64
 8   2p        205768 non-null  float64
 9   2pa       205768 non-null  float64
 10  2p_perc   205765 non-null  float64
 11  3p        205768 non-null  float64
 12  3pa       205768 non-null  float64
 13  3p_perc   204304 non-null  float64
 14  efg_perc  205768 non-null  float64
 15  ast       205768 non-null  float64
 16  ast_perc  205745 non-null  float64
dtypes: float64(12), int64(1), object(4)
memory usage: 26.7+ MB


In [5]:
df.head()

,date,visitor,home,team,quarter,fg,fga,fg_perc,2p,2pa,2p_perc,3p,3pa,3p_perc,efg_perc,ast,ast_perc
0,"Tue, Oct 31, 2006",Chicago Bulls,Miami Heat,0,q1,5.0,20.0,0.250,5.0,16.0,0.313,0.0,4.0,0.000,0.250,3.0,0.600
1,"Tue, Oct 31, 2006",Chicago Bulls,Miami Heat,0,q2,15.0,19.0,0.789,12.0,16.0,0.750,3.0,3.0,1.000,0.868,10.0,0.667
2,"Tue, Oct 31, 2006",Chicago Bulls,Miami Heat,0,q3,8.0,21.0,0.381,5.0,16.0,0.313,3.0,5.0,0.600,0.452,4.0,0.500
3,"Tue, Oct 31, 2006",Chicago Bulls,Miami Heat,0,q4,11.0,19.0,0.579,10.0,18.0,0.556,1.0,1.0,1.000,0.605,5.0,0.455
4,"Tue, Oct 31, 2006",Chicago Bulls,Miami Heat,0,total,39.0,79.0,0.494,32.0,66.0,0.485,7.0,13.0,0.538,0.538,22.0,0.564


In [6]:
# Convert 'date' column to Date object
df['date'] = pd.to_datetime(df['date'])


# Conver 'team' column to Team Name
df['team'] = np.where(df['team'], df['home'], df['visitor'])


# Merge in opponents (see team defensive stats)
df = pd.merge(
    df, 
    df, 
    left_on=['date', 'visitor', 'home', 'quarter'], 
    right_on=['date', 'visitor', 'home', 'quarter'],
    suffixes=('', '_opp'),
    how='left')

df = df[df['team'] != df['team_opp']]


df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 205810 entries, 1 to 411618
Data columns (total 30 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   date          205810 non-null  datetime64[ns]
 1   visitor       205810 non-null  object        
 2   home          205810 non-null  object        
 3   team          205810 non-null  object        
 4   quarter       205788 non-null  object        
 5   fg            205768 non-null  float64       
 6   fga           205768 non-null  float64       
 7   fg_perc       205768 non-null  float64       
 8   2p            205768 non-null  float64       
 9   2pa           205768 non-null  float64       
 10  2p_perc       205765 non-null  float64       
 11  3p            205768 non-null  float64       
 12  3pa           205768 non-null  float64       
 13  3p_perc       204304 non-null  float64       
 14  efg_perc      205768 non-null  float64       
 15  ast           205

In [7]:
total_df = df[df['quarter'] == 'total']
q1_df = df[df['quarter'] == 'q1']
q2_df = df[df['quarter'] == 'q2']
q3_df = df[df['quarter'] == 'q3']
q4_df = df[df['quarter'] == 'q4']

In [8]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40608 entries, 9 to 411618
Data columns (total 30 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          40608 non-null  datetime64[ns]
 1   visitor       40608 non-null  object        
 2   home          40608 non-null  object        
 3   team          40608 non-null  object        
 4   quarter       40608 non-null  object        
 5   fg            40588 non-null  float64       
 6   fga           40588 non-null  float64       
 7   fg_perc       40588 non-null  float64       
 8   2p            40588 non-null  float64       
 9   2pa           40588 non-null  float64       
 10  2p_perc       40588 non-null  float64       
 11  3p            40588 non-null  float64       
 12  3pa           40588 non-null  float64       
 13  3p_perc       40588 non-null  float64       
 14  efg_perc      40588 non-null  float64       
 15  ast           40588 non-null  float

# Dataframe of team's last 15 performances

In [9]:
# Totals
total_df = df[df['quarter'] == 'total'].copy()

# Return ten lastest dates team played
def last_15_date(team, date):
    schedule = total_df[total_df['team'] == team].sort_values(by='date').reset_index()
    date_index = schedule[schedule['date'] == date].index[0]
    if date_index - 15 < 0:
        return None, None, None, None, None, None, None, None, None, None, None, None, None, None, None
    else:
        date_1, date_2 = schedule.iloc[date_index - 1]['date'], schedule.iloc[date_index - 2]['date']
        date_3, date_4 = schedule.iloc[date_index - 3]['date'], schedule.iloc[date_index - 4]['date']
        date_5, date_6 = schedule.iloc[date_index - 5]['date'], schedule.iloc[date_index - 6]['date']
        date_7, date_8 = schedule.iloc[date_index - 7]['date'], schedule.iloc[date_index - 8]['date']
        date_9, date_10 = schedule.iloc[date_index - 9]['date'], schedule.iloc[date_index - 10]['date']
        date_11, date_12 = schedule.iloc[date_index - 11]['date'], schedule.iloc[date_index - 12]['date']
        date_13, date_14 = schedule.iloc[date_index - 13]['date'], schedule.iloc[date_index - 14]['date']
        date_15 = schedule.iloc[date_index - 15]['date']
        return date_1, date_2, date_3, date_4, date_5, date_6, date_7, date_8, date_9, date_10, date_11, date_12, date_13, date_14, date_15

total_df['dates'] = total_df.apply(lambda x: last_15_date(x.team, x.date), axis=1)
total_df['date_1'], total_df['date_2'] = total_df['dates'].apply(lambda x: x[0]), total_df['dates'].apply(lambda x: x[1])
total_df['date_3'], total_df['date_4'] = total_df['dates'].apply(lambda x: x[2]), total_df['dates'].apply(lambda x: x[3])
total_df['date_5'], total_df['date_6'] = total_df['dates'].apply(lambda x: x[4]), total_df['dates'].apply(lambda x: x[5])
total_df['date_7'], total_df['date_8'] = total_df['dates'].apply(lambda x: x[6]), total_df['dates'].apply(lambda x: x[7])
total_df['date_9'], total_df['date_10'] = total_df['dates'].apply(lambda x: x[8]), total_df['dates'].apply(lambda x: x[9])
total_df['date_11'], total_df['date_12'] = total_df['dates'].apply(lambda x: x[10]), total_df['dates'].apply(lambda x: x[11])
total_df['date_13'], total_df['date_14'] = total_df['dates'].apply(lambda x: x[12]), total_df['dates'].apply(lambda x: x[13])
total_df['date_15'] = total_df['dates'].apply(lambda x: x[14])

In [10]:
# Define statistics
stats = ['fg', 'fga', '2p', '2pa', '3p', '3pa', 'ast',
         'fg_opp', 'fga_opp', '2p_opp', '2pa_opp', '3p_opp', '3pa_opp', 'ast_opp']

perc_stats = ['fg_perc', '2p_perc', '3p_perc', 'efg_perc', 'ast_perc',
              'fg_perc_opp', '2p_perc_opp', '3p_perc_opp', 'efg_perc_opp', 'ast_perc_opp']

# X and y column names to merge on
x_cols = ['date', 'team'] + stats

last_15_games = total_df.copy()
last_15_games.loc[:, 'target'] = last_15_games.loc[:, '3p']
X = total_df[x_cols].copy()

# Dataframe of target (3pt made by each team) and of variables (last 5 games stats for each team)
dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']
for date in dates:
    last_15_games = pd.merge(last_15_games, X, left_on=['date' + date, 'team'], right_on=['date', 'team'], how='left', suffixes=('', date))

### Last 15 Performances (Unweighted)

In [11]:
# Define statistics
stats = ['fg', 'fga', '2p', '2pa', '3p', '3pa', 'ast',
         'fg_opp', 'fga_opp', '2p_opp', '2pa_opp', '3p_opp', '3pa_opp', 'ast_opp']

perc_stats = ['fg_perc', '2p_perc', '3p_perc', 'efg_perc', 'ast_perc',
              'fg_perc_opp', '2p_perc_opp', '3p_perc_opp', 'efg_perc_opp', 'ast_perc_opp']

dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']

last_15 = last_15_games.copy()

# Calculate mean for each stat over a team's last performance
for stat in stats:
    last_15[stat + '_last_15'] = 0
    last_15[stat + '_last_3'] = 0
    last_15[stat + '_last_1'] = 0
    
    for date in dates:
        # Last 15 games
        last_15[stat + '_last_15'] = last_15[stat + '_last_15'] + last_15[stat + date]
        
        # Last 3 games
        if date in ['_1', '_2', '_3']:
            last_15[stat + '_last_3'] = last_15[stat + '_last_3'] + last_15[stat + date]
        
        # Last game
        if date in ['_1']:
            last_15[stat + '_last_1'] = last_15[stat + '_last_1'] + last_15[stat + date]
        
    
    last_15[stat + '_last_15'] = last_15[stat + '_last_15'] / 15
    last_15[stat + '_last_3'] = last_15[stat + '_last_3'] / 3
    
# Calculate difference between last 15 games, 3 games and last game
for stat in stats:
    last_15[stat + '_trend_3'] = last_15[stat + '_last_15'] - last_15[stat + '_last_3']
    last_15[stat + '_trend_1'] = last_15[stat + '_last_15'] - last_15[stat + '_last_1']

# Sum stats for opposing teams for each game
last_15 = last_15.groupby(['date', 'visitor', 'home']).sum()

# Percentages for matchup
for perc in perc_stats:
    stat = perc.split('_')[0]
    opp = perc.split('_')[-1]
    if opp == 'opp':
        if stat == 'ast':
            last_15[perc + '_last_15'] = last_15[stat + '_opp_last_15'] / last_15['fg_opp_last_15']
            last_15[perc + '_last_3'] = last_15[stat + '_opp_last_3'] / last_15['fg_opp_last_3']
            last_15[perc + '_last_1'] = last_15[stat + '_opp_last_1'] / last_15['fg_opp_last_1']
        elif stat == 'efg':
            last_15[perc + '_last_15'] = (last_15['fg_opp_last_15'] + (.5 * last_15['3p_opp_last_15'])) / last_15['fga_opp_last_15']
            last_15[perc + '_last_3'] = (last_15['fg_opp_last_3'] + (.5 * last_15['3p_opp_last_3'])) / last_15['fga_opp_last_3']
            last_15[perc + '_last_1'] = (last_15['fg_opp_last_1'] + (.5 * last_15['3p_opp_last_1'])) / last_15['fga_opp_last_1']
        else:
            last_15[perc + '_last_15'] = last_15[stat + '_opp_last_15'] / last_15[stat + 'a_opp_last_15']
            last_15[perc + '_last_3'] = last_15[stat + '_opp_last_3'] / last_15[stat + 'a_opp_last_3']
            last_15[perc + '_last_1'] = last_15[stat + '_opp_last_1'] / last_15[stat + 'a_opp_last_1']
    else:
        if stat == 'ast':
            last_15[perc + '_last_15'] = last_15[stat + '_last_15'] / last_15['fg' + '_last_15']
            last_15[perc + '_last_3'] = last_15[stat + '_last_3'] / last_15['fg' + '_last_3']
            last_15[perc + '_last_1'] = last_15[stat + '_last_1'] / last_15['fg' + '_last_1']
        elif stat == 'efg':
            last_15[perc + '_last_15'] = (last_15['fg' + '_last_15'] + (.5 * last_15['3p' + '_last_15'])) / last_15['fga' + '_last_15']
            last_15[perc + '_last_3'] = (last_15['fg' + '_last_3'] + (.5 * last_15['3p' + '_last_3'])) / last_15['fga' + '_last_3']
            last_15[perc + '_last_1'] = (last_15['fg' + '_last_1'] + (.5 * last_15['3p' + '_last_1'])) / last_15['fga' + '_last_1']
        else:
            last_15[perc + '_last_15'] = last_15[stat + '_last_15'] / last_15[stat + 'a_last_15']
            last_15[perc + '_last_3'] = last_15[stat + '_last_3'] / last_15[stat + 'a_last_3']
            last_15[perc + '_last_1'] = last_15[stat + '_last_1'] / last_15[stat + 'a_last_1']

# Keep columns
stats_15 = [stat + '_last_15' for stat in stats]
stats_3 = [stat + '_last_3' for stat in stats]
stats_1 = [stat + '_last_1' for stat in stats]
trend_3 = [stat + '_trend_3' for stat in stats]
trend_1 = [stat + '_trend_1' for stat in stats]
last_15 = last_15[['target'] + stats_1 + stats_3 + stats_15 + perc_stats + trend_1 + trend_3]
        
last_15 = last_15.dropna(axis=0)
last_15.tail(10)

target  fg_last_1  \
date       visitor                home                                       
2022-04-01 Dallas Mavericks       Washington Wizards        0.0       93.0   
           Detroit Pistons        Oklahoma City Thunder     0.0       82.0   
           Indiana Pacers         Boston Celtics            0.0       79.0   
           Los Angeles Clippers   Milwaukee Bucks           0.0       85.0   
           Minnesota Timberwolves Denver Nuggets            0.0       87.0   
           New Orleans Pelicans   Los Angeles Lakers        0.0       81.0   
           Phoenix Suns           Memphis Grizzlies         0.0       82.0   
           Portland Trail Blazers San Antonio Spurs         0.0       81.0   
           Sacramento Kings       Houston Rockets           0.0       87.0   
           Toronto Raptors        Orlando Magic             0.0       88.0   

                                                         fga_last_1  \
date       visitor                home                                
2022-04-01 Dallas Mavericks       Washington Wizards          180.0   
           Detroit Pistons        Oklahoma City Thunder       169.0   
           Indiana Pacers         Boston Celtics              182.0   
           Los Angeles Clippers   Milwaukee Bucks             179.0   
           Minnesota Timberwolves Denver Nuggets              172.0   
           New Orleans Pelicans   Los Angeles Lakers          161.0   
           Phoenix Suns           Memphis Grizzlies           183.0   
           Portland Trail Blazers San Antonio Spurs           168.0   
           Sacramento Kings       Houston Rockets             189.0   
           Toronto Raptors        Orlando Magic               192.0   

                                                         2p_last_1  \
date       visitor                home                               
2022-04-01 Dallas Mavericks       Washington Wizards          64.0   
           Detroit Pistons        Oklahoma City Thunder       55.0   
           Indiana Pacers         Boston Celtics              56.0   
           Los Angeles Clippers   Milwaukee Bucks             57.0   
           Minnesota Timberwolves Denver Nuggets              64.0   
           New Orleans Pelicans   Los Angeles Lakers          68.0   
           Phoenix Suns           Memphis Grizzlies           59.0   
           Portland Trail Blazers San Antonio Spurs           53.0   
           Sacramento Kings       Houston Rockets             67.0   
           Toronto Raptors        Orlando Magic               59.0   

                                                         2pa_last_1  \
date       visitor                home                                
2022-04-01 Dallas Mavericks       Washington Wizards          102.0   
           Detroit Pistons        Oklahoma City Thunder        93.0   
           Indiana Pacers         Boston Celtics              105.0   
           Los Angeles Clippers   Milwaukee Bucks             102.0   
           Minnesota Timberwolves Denver Nuggets              107.0   
           New Orleans Pelicans   Los Angeles Lakers          112.0   
           Phoenix Suns           Memphis Grizzlies           116.0   
           Portland Trail Blazers San Antonio Spurs            91.0   
           Sacramento Kings       Houston Rockets             121.0   
           Toronto Raptors        Orlando Magic               123.0   

                                                         3p_last_1  \
date       visitor                home                               
2022-04-01 Dallas Mavericks       Washington Wizards          29.0   
           Detroit Pistons        Oklahoma City Thunder       27.0   
           Indiana Pacers         Boston Celtics              23.0   
           Los Angeles Clippers   Milwaukee Bucks             28.0   
           Minnesota Timberwolves Denver Nuggets              23.0   
           New Orleans Pelicans   Los Angeles Lakers          13.0   
           Ph

## Correlations

In [12]:
corr_df = pd.DataFrame()

# Correlations for last 15 game stats vs 3pt made (unweighted)
for col in last_15:
    corr_p = pearsonr(last_15['target'], last_15[col])
    row = {'stat': col, 'corr': round(corr_p[0], 2), 'p-value': round(corr_p[1], 2)}
    corr_df = corr_df.append(row, ignore_index=True)
    
# Print correlation
corr_df = corr_df[corr_df['p-value'] < .05].drop(['p-value'], axis=1).sort_values(['corr'], axis=0, ascending=False)
corr_df

,corr,stat
0,1.00,target
34,0.72,3pa_last_15
33,0.72,3p_last_15
20,0.71,3pa_last_3
6,0.68,3pa_last_1
...,...,...
25,-0.20,2pa_opp_last_3
39,-0.21,2pa_opp_last_15
4,-0.27,2pa_last_1
18,-0.29,2pa_last_3


## Save dataframe with significantly correlated stats

In [13]:
stats = corr_df[corr_df['corr'].abs() >= .6]['stat']
df = last_15[stats]

df.to_csv('backend/data/inputs/3p/shooting.csv')